In [1]:
import bs4 as bs
import pickle
import requests
import pandas as pd
import json
from dateutil import parser
from datetime import datetime
import os
import time
from pandas.io.json import json_normalize
from alpha_vantage.timeseries import TimeSeries
from alpha_vantage.fundamentaldata import FundamentalData
import wrds
from datetime import datetime as dt
import numpy as np
import glob, os
import csv

pd.set_option("display.precision", 8)


## Tickers

> ### Gets currently listed SP500 companies and thier CIKs

In [2]:
# gets list of SP500 companies and their CIKs

#standard: pull from yahoo finance, yfinance
#

def save_SP500_tickers():
    resp = requests.get('https://web.archive.org/web/20200411084847/https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
    soup = bs.BeautifulSoup(resp.text, 'lxml')
    table = soup.find('table', {'class': 'wikitable sortable'})
    tickers = []
    ciks = []
    
    for row in table.findAll('tr')[1:]:
        ticker = row.findAll('td')[0].text[:-1]
        cik = row.findAll('td')[7].text[:]
        
        tickers.append(ticker)
        ciks.append(cik)
    
    #drops '\n' from certain entries in CIK list
    ciks = list(map(lambda x:x.strip(),ciks))
    
    tickers_ciks_df = pd.DataFrame({'ticker': tickers, 'cik': ciks})

        
    return tickers_ciks_df

> ### Prepping tickers list

> ### Remove tickers from list if change detected in change list

In [3]:
fundamentals_start = pd.to_datetime('2009-02-09', format = '%Y-%m-%d')
fundamentals_end = pd.to_datetime('2020-02-03', format = '%Y-%m-%d')

In [4]:
start_date = '2009-06-30'#pd.to_datetime('2009-06-02', format = '%Y-%m-%d') # Define start and end dates
end_date = '2020-02-03'#pd.to_datetime('2020-04-15', format = '%Y-%m-%d')

In [5]:
changes = pd.read_csv('S&P 500 Historical Components & Changes(10-14-2022).csv',
                              header=0,
                              index_col=0,
                              parse_dates=True)

C:\Users\Daniel\AppData\Local\Temp\ipykernel_13888\2208461315.py:1: DtypeWarning: Columns (500,501,502,503,504,505,506,507) have mixed types. Specify dtype option on import or set low_memory=False.
  changes = pd.read_csv('S&P 500 Historical Components & Changes(10-14-2022).csv',


In [6]:
tickers = sorted(set.intersection(*list(changes[start_date:end_date].apply(lambda x: set(x.dropna()), axis=1))))

In [7]:
len(tickers)

312

## Financial Modelling Company Quarterly Financials

> ### Get Income Statements and Balance Sheets

In [8]:
fpc_key = 'xxxxxxxxx'
n_months = 58

In [9]:
'''
income_dfs = []
for company in tickers:
    r = requests.get(f"https://financialmodelingprep.com/api/v3/income-statement/{company}?period=quarter&limit={n_months}&apikey={fpc_key}")
    r= r.json()
    data = pd.DataFrame.from_dict(r)
    income_dfs.append(data)
#'''

'\nincome_dfs = []\nfor company in tickers:\n    r = requests.get(f"https://financialmodelingprep.com/api/v3/income-statement/{company}?period=quarter&limit={n_months}&apikey={fpc_key}")\n    r= r.json()\n    data = pd.DataFrame.from_dict(r)\n    income_dfs.append(data)\n#'

In [10]:
'''
balance_dfs = []
for company in tickers:
    r = requests.get(f"https://financialmodelingprep.com/api/v3/balance-sheet-statement/{company}?period=quarter&limit={n_months}&apikey={fpc_key}")
    r= r.json()
    data = pd.DataFrame.from_dict(r)
    balance_dfs.append(data)
#'''

'\nbalance_dfs = []\nfor company in tickers:\n    r = requests.get(f"https://financialmodelingprep.com/api/v3/balance-sheet-statement/{company}?period=quarter&limit={n_months}&apikey={fpc_key}")\n    r= r.json()\n    data = pd.DataFrame.from_dict(r)\n    balance_dfs.append(data)\n#'

> ### Dump or Load Quarterly Data to Pickle

In [11]:
#dump to pickle
'''
with open('quarterly_income', 'wb') as fh:
    pickle.dump(income_dfs, fh)
    
with open('quarterly_balance', 'wb') as fh:
    pickle.dump(balance_dfs, fh)
#'''

"\nwith open('quarterly_income', 'wb') as fh:\n    pickle.dump(income_dfs, fh)\n    \nwith open('quarterly_balance', 'wb') as fh:\n    pickle.dump(balance_dfs, fh)\n#"

In [12]:
# load from pickle

#'''
with open("quarterly_income","rb") as fin:
    income_dfs = pickle.load(fin)


with open("quarterly_balance","rb") as fin:
    balance_dfs = pickle.load(fin)
#'''


> ### Get Quarterly Net Income, Revenue and Total Assets 

> > #### Cleaned Income Statement Data

In [13]:
clean_income_dfs = []
cols_to_keep = ['date', 'symbol', 'reportedCurrency','fillingDate','period', 'revenue','netIncome']
for c, company in enumerate(income_dfs):
    if 'date' in list(company.columns):
        company = company[cols_to_keep]
        company = company.set_index('date')
        company = company.iloc[::-1]
        clean_income_dfs.append(company)
        
    elif 'date' not in list(company.columns):
        tickers.pop(c)
        balance_dfs.pop(c)

> > #### Cleaned Income Statement Data

In [14]:
clean_balance_dfs = []
cols_to_keep = ['date', 'symbol', 'reportedCurrency','fillingDate','period', 'totalAssets']
for company in balance_dfs:
    company = company[cols_to_keep]
    company = company.set_index('date')
    company = company.iloc[::-1]
    clean_balance_dfs.append(company)

### > ### Drop Companies with Missing Data

> > #### Balance Sheet

In [15]:

balance_not_dropped = []


for df in clean_balance_dfs:
    df.index = pd.to_datetime(df.index, format = '%Y-%m-%d')
    df['difference'] = df.index.to_series().diff().dt.days
    balance_not_dropped.append(pd.DataFrame(df))
   


In [16]:


clean_balance_dfs_not_dropped = []

for df in balance_not_dropped:
    if df['difference'].bfill().min() > 75:
        df['fillingDate'] = pd.to_datetime(df['fillingDate'], format='%Y-%m-%d', errors='ignore')
        clean_balance_dfs_not_dropped.append(df[:-1])
        
        

In [17]:
#clean_balance_dfs_not_dropped.insert(0, clean_balance_dfs_not_dropped.pop(167))

balance_date_matched = []

#clean_balance_dfs.sort(key=lambda df: df.index.min())

for count, df in enumerate(clean_balance_dfs_not_dropped):

        
    if count>0 and count < len(clean_balance_dfs_not_dropped)-1:
        
        
        merged_dataframe = pd.merge_asof(clean_balance_dfs_not_dropped[0],
                                         clean_balance_dfs_not_dropped[count+1],
                                        on='date')
        balance = merged_dataframe.iloc[:, np.r_[0,7:12]]
        balance = balance.set_index('date')
        balance = balance.rename(columns = lambda x : str(x)[:-2]) 
        balance_date_matched.append(balance)

        balance = []
        merged_dataframe = []
        
    if count == len(clean_balance_dfs_not_dropped)-1:

        merged_dataframe = pd.merge_asof(clean_balance_dfs_not_dropped[0],
                                         clean_balance_dfs_not_dropped[count],
                                        on='date')
        balance = merged_dataframe.iloc[:, np.r_[0,7:12]]
        balance = balance.set_index('date')
        balance = balance.rename(columns = lambda x : str(x)[:-2]) 
        balance_date_matched.append(balance)

        #balance = []
        #merged_dataframe = []


    

In [18]:
trimmed_balance = []
for df in balance_date_matched:
    x = df["2009-09-01":"2019-10-15"]
    
    if not x.isna().any().any():
        trimmed_balance.append(x)
        
trimmed_balance = trimmed_balance[:-1]

In [19]:
for df in trimmed_balance:
    print(df.iloc[0,0])

ABC
ABT
ADBE
ADI
ADM
ADP
ADSK
AEE
AEP
AES
AFL
AIG
AIV
AIZ
AKAM
ALL
AMAT
AMGN
AMP
AMT
AMZN
ANTM
AON
APA
APD
APH
AVB
AVY
AXP
AZO
BA
BAC
BAX
BDX
BEN
BIIB
BK
BLL
BMY
BSX
BXP
C
CAG
CAH
CAT
CB
CBRE
CCL
CF
CHRW
CI
CINF
CL
CLX
CMA
CMCSA
CME
CMI
CMS
CNP
COF
COG
COP
CPB
CRM
CSCO
CSX
CTAS
CTL
CTSH
CTXS
CVS
CVX
D
DE
DFS
DGX
DHI
DHR
DIS
DOV
DRI
DTE
DUK
DVA
DVN
EA
EBAY
ECL
ED
EFX
EIX
EL
EMN
EMR
EOG
EQR
ES
ETFC
ETN
ETR
EXC
EXPD
EXPE
F
FAST
FCX
FDX
FE
FIS
FISV
FITB
FLIR
FLS
FTI
GD
GE
GILD
GIS
GLW
GOOGL
GPC
GPS
GS
GWW
HAL
HAS
HBAN
HD
HES
HIG
HOG
HON
HPQ
HRB
HRL
HST
HSY
HUM
IBM
ICE
IFF
INTC
INTU
IP
IPG
IRM
ISRG
ITW
IVZ
JCI
JNJ
JNPR
JPM
JWN
K
KEY
KIM
KLAC
KMB
KO
KR
KSS
L
LEG
LEN
LH
LLY
LMT
LNC
LOW
LUV
M
MA
MAR
MAS
MCD
MCHP
MCK
MCO
MDLZ
MDT
MET
MKC
MMC
MMM
MO
MRK
MRO
MS
MSFT
MSI
MTB
MU
NBL
NDAQ
NEE
NEM
NI
NKE
NOC
NOV
NSC
NTAP
NTRS
NUE
NVDA
NWL
OMC
ORCL
ORLY
OXY
PAYX
PBCT
PCAR
PEG
PEP
PFE
PFG
PG
PGR
PH
PHM
PKI
PLD
PM
PNC
PNW
PPG
PPL
PRU
PSA
PXD
QCOM
RF
RL
ROK
RSG
RTN
SBUX
SCHW
SEE
SHW
SJM
SLB
SNA
SO
SPG
SP

> > #### Income Statement

In [20]:
income_not_dropped = []

for df in clean_income_dfs:
    df.index = pd.to_datetime(df.index, format = '%Y-%m-%d')
    df['difference'] = df.index.to_series().diff().dt.days
    income_not_dropped.append(pd.DataFrame(df))

In [21]:
clean_income_dfs_not_dropped = []

for df in income_not_dropped:
    if df['difference'].bfill().min() > 75:
        df['fillingDate'] = pd.to_datetime(df['fillingDate'], format='%Y-%m-%d', errors='ignore')
        clean_income_dfs_not_dropped.append(df[:-1])

In [22]:
#clean_balance_dfs_not_dropped.insert(0, clean_balance_dfs_not_dropped.pop(167))

income_date_matched = []

#clean_balance_dfs.sort(key=lambda df: df.index.min())

for count, df in enumerate(clean_income_dfs_not_dropped):

        
    if count>0 and count < len(clean_income_dfs_not_dropped)-1:
        
        
        merged_dataframe = pd.merge_asof(clean_income_dfs_not_dropped[0],
                                         clean_income_dfs_not_dropped[count+1],
                                        on='date')
        income = merged_dataframe.iloc[:, np.r_[0,8:14]]
        income = income.set_index('date')
        income = income.rename(columns = lambda x : str(x)[:-2]) 
        income_date_matched.append(income)

        income = []
        merged_dataframe = []
        
    if count == len(clean_income_dfs_not_dropped)-1:

        merged_dataframe = pd.merge_asof(clean_income_dfs_not_dropped[0],
                                         clean_income_dfs_not_dropped[count],
                                        on='date')
        income = merged_dataframe.iloc[:, np.r_[0,6:14]]
        income = income.set_index('date')
        income = income.rename(columns = lambda x : str(x)[:-2]) 
        income_date_matched.append(income)

        #income = []
        #merged_dataframe = []


    

In [23]:
trimmed_income = []
for df in income_date_matched:
    x = df["2009-09-01":"2019-10-15"]
    #x.netIncome.values.astype(int)
    if not x.isna().any().any():
        trimmed_income.append(x)
trimmed_income = trimmed_income[:-1]

In [24]:
trimmed_balance[0]

,symbol,reportedCurrency,fillingDate,period,totalAssets
date,,,,,
2009-10-31,ABC,USD,2009-11-25,Q4,13572740000.0
2010-01-31,ABC,USD,2010-02-05,Q1,13556271000.0
2010-04-30,ABC,USD,2010-05-07,Q2,13822288000.0
2010-07-31,ABC,USD,2010-08-03,Q3,14003785000.0
2010-10-31,ABC,USD,2010-11-23,Q4,14434843000.0
2011-01-31,ABC,USD,2011-02-08,Q1,14132319000.0
2011-04-30,ABC,USD,2011-05-06,Q2,14681016000.0
2011-07-31,ABC,USD,2011-08-08,Q3,14855729000.0
2011-10-31,ABC,USD,2011-11-22,Q4,14982671000.0


In [25]:
for df in trimmed_balance:
    print(df.shape)

(40, 5)
(40, 5)
(40, 5)
(40, 5)
(40, 5)
(40, 5)
(40, 5)
(40, 5)
(40, 5)
(40, 5)
(40, 5)
(40, 5)
(40, 5)
(40, 5)
(40, 5)
(40, 5)
(40, 5)
(40, 5)
(40, 5)
(40, 5)
(40, 5)
(40, 5)
(40, 5)
(40, 5)
(40, 5)
(40, 5)
(40, 5)
(40, 5)
(40, 5)
(40, 5)
(40, 5)
(40, 5)
(40, 5)
(40, 5)
(40, 5)
(40, 5)
(40, 5)
(40, 5)
(40, 5)
(40, 5)
(40, 5)
(40, 5)
(40, 5)
(40, 5)
(40, 5)
(40, 5)
(40, 5)
(40, 5)
(40, 5)
(40, 5)
(40, 5)
(40, 5)
(40, 5)
(40, 5)
(40, 5)
(40, 5)
(40, 5)
(40, 5)
(40, 5)
(40, 5)
(40, 5)
(40, 5)
(40, 5)
(40, 5)
(40, 5)
(40, 5)
(40, 5)
(40, 5)
(40, 5)
(40, 5)
(40, 5)
(40, 5)
(40, 5)
(40, 5)
(40, 5)
(40, 5)
(40, 5)
(40, 5)
(40, 5)
(40, 5)
(40, 5)
(40, 5)
(40, 5)
(40, 5)
(40, 5)
(40, 5)
(40, 5)
(40, 5)
(40, 5)
(40, 5)
(40, 5)
(40, 5)
(40, 5)
(40, 5)
(40, 5)
(40, 5)
(40, 5)
(40, 5)
(40, 5)
(40, 5)
(40, 5)
(40, 5)
(40, 5)
(40, 5)
(40, 5)
(40, 5)
(40, 5)
(40, 5)
(40, 5)
(40, 5)
(40, 5)
(40, 5)
(40, 5)
(40, 5)
(40, 5)
(40, 5)
(40, 5)
(40, 5)
(40, 5)
(40, 5)
(40, 5)
(40, 5)
(40, 5)
(40, 5)
(40, 5)


> ### Calculate Return on Assets and Asset Turnover

In [26]:
ratio_dfs = []

for company, c in enumerate(trimmed_balance):
    


    merged = pd.merge(trimmed_income[int(company)], trimmed_balance[int(company)],
                          on=['date', 'symbol', 'reportedCurrency','fillingDate', 'period'])
    ratios = pd.DataFrame()
    ratios['RoA'] = merged[['netIncome']].div(merged.totalAssets, axis = 0)
    ratios['AT'] = merged[['revenue']].div(merged.totalAssets, axis = 0)
    
    #reverse dataframes so date go in ascending order
    #merged = merged.iloc[::-1]
    
    df = np.concatenate([ratios.RoA.values,ratios.AT.values])
    df = pd.DataFrame(df, columns = [c.iloc[0]['symbol']])
    #df = df.set
        #df.columns = c.iloc[0]['symbol']
    ratio_dfs.append(df)

In [27]:
ratio_dfs[0]

,ABC
0,0.00958885
1,0.01116140
2,0.01309537
3,0.01165435
4,0.00978383
...,...
75,1.12614901
76,1.14938471
77,1.15597068
78,1.11618799


> ### Format Ratios Data 

In [28]:
#final list of ratio dfs.
#RoA and AT are concatenated together vertically as a single list. 
#41 quarters of data for each company
ratios_final =[]
for df in ratio_dfs:
    
    if df.shape[0] == 80:
        ratios_final.append(df)

> ### Export Ratios as CSV

In [29]:
ratios_export = pd.concat(ratios_final, axis=1).T.dropna(how='all',axis=0).dropna(how='all',axis=1)

In [30]:
ratios_export.shape

(296, 80)

In [31]:
ratios_export.replace([np.inf, -np.inf], np.nan, inplace=True)

In [32]:
ratios_export.dropna(how="any", inplace=True)

In [33]:
ratios_export.shape

(296, 80)

In [34]:
ratios_export[:-1].to_csv('Export/ratios.csv')